## Set up the enivornment

In [ ]:
pip install cassandra-driver pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 31.2 MB/s eta 0:00:00


In [ ]:
import cassandra
print(cassandra.__version__)


3.29.2


## Creating the directory

In [ ]:
import os

# Create a directory named "Assignment3"
os.mkdir("Assignment3")


In [ ]:
import os

# Change to the "python_project" directory
os.chdir("Assignment3")

# Verify the current working directory
print("Current Directory:", os.getcwd())

Current Directory: /content/Assignment3


In [ ]:
with open("connect_database.py", "w") as file:
    pass

## Creating connection to cassendra.

In [10]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json

# This secure connect bundle is autogenerated when you download your SCB,
# if yours is different update the file name below
cloud_config = {
    'secure_connect_bundle': '/content/Assignment3/secure-connect-sneha.zip'
}
# This token JSON file is autogenerated when you download your token,
# if yours is different update the file name below
with open("/content/Assignment3/Sneha-token.json") as f:
    secrets = json.load(f)


CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

row = session.execute("select release_version from system.local").one()
if row:
  print(row[0])
else:
  print("An error occurred.")

4.0.0.6816


In [11]:
!python ./connect_database.py

## creating bronze level table

In [12]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import csv
from datetime import datetime
import uuid

# Connection setup
client_id = 'XTHZjgZRWbSZqXvWcYmtJAud'
client_secret = 'qh-ElzlZAG_bfeoDjZ9.ZCbht7jJ_WrQrh27Se6RsPNFxgz6wBnlY+B2KkyCXpwiMna.-F_bQy1I7Le5.CByML5,1MjuoWRNTjYYjl8NbrcwI1Yqrq2G2Q.u48XSY0Km'
secure_connect_bundle_path = '/content/Assignment3/secure-connect-sneha.zip'

# Set up authentication
auth_provider = PlainTextAuthProvider(client_id, client_secret)
cluster = Cluster(
    cloud={'secure_connect_bundle': secure_connect_bundle_path},
    auth_provider=auth_provider,
    protocol_version=4
)

try:
    session = cluster.connect()
    session.set_keyspace('sneha')

    # Drop existing table if it exists
    session.execute("DROP TABLE IF EXISTS bronze_sales")

    # Create Bronze Table
    create_table_query = """
    CREATE TABLE IF NOT EXISTS bronze_sales (
        id UUID PRIMARY KEY,
        Region TEXT,
        Country TEXT,
        Item_Type TEXT,
        Sales_Channel TEXT,
        Order_Priority TEXT,
        Order_Date DATE,
        Order_ID TEXT,
        Ship_Date DATE,
        Units_Sold INT,
        Unit_Price DECIMAL,
        Unit_Cost DECIMAL,
        Total_Revenue DECIMAL,
        Total_Cost DECIMAL,
        Total_Profit DECIMAL
    )
    """
    session.execute(create_table_query)
    print("Bronze table created successfully")

    # Function to parse date
    def parse_date(date_str):
        return datetime.strptime(date_str, '%m/%d/%Y').date()

    # Read and insert data from CSV
    csv_file_path = '/content/Assignment3/sales_100.csv'
    with open(csv_file_path, 'r') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            try:
                insert_query = """
                INSERT INTO bronze_sales (
                    id, Region, Country, Item_Type, Sales_Channel,
                    Order_Priority, Order_Date, Order_ID, Ship_Date, Units_Sold,
                    Unit_Price, Unit_Cost, Total_Revenue, Total_Cost, Total_Profit
                )
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """
                session.execute(insert_query, (
                    uuid.uuid4(),
                    row['Region'],
                    row['Country'],
                    row['Item Type'],
                    row['Sales Channel'],
                    row['Order Priority'],
                    parse_date(row['Order Date']),
                    row['Order ID'],
                    parse_date(row['Ship Date']),
                    int(row['UnitsSold']),  # Changed from 'Units Sold' to 'UnitsSold'
                    float(row['UnitPrice']), # Changed from 'Unit Price' to 'UnitPrice'
                    float(row['UnitCost']),  # Changed from 'Unit Cost' to 'UnitCost'
                    float(row['TotalRevenue']), # Changed from 'Total Revenue' to 'TotalRevenue'
                    float(row['TotalCost']),    # Changed from 'Total Cost' to 'TotalCost'
                    float(row['TotalProfit'])   # Changed from 'Total Profit' to 'TotalProfit'
                ))
                print(f"Inserted row with Order ID: {row['Order ID']}")
            except Exception as e:
                print(f"Error inserting row with Order ID {row['Order ID']}: {e}")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    cluster.shutdown()
    print("Connection closed")

Bronze table created successfully
Inserted row with Order ID: 443368995
Inserted row with Order ID: 667593514
Inserted row with Order ID: 940995585
Inserted row with Order ID: 880811536
Inserted row with Order ID: 174590194
Inserted row with Order ID: 830192887
Inserted row with Order ID: 425793445
Inserted row with Order ID: 659878194
Inserted row with Order ID: 601245963
Inserted row with Order ID: 739008080
Inserted row with Order ID: 732588374
Inserted row with Order ID: 761723172
Inserted row with Order ID: 176461303
Inserted row with Order ID: 647164094
Inserted row with Order ID: 314505374
Inserted row with Order ID: 539471471
Inserted row with Order ID: 953361213
Inserted row with Order ID: 807785928
Inserted row with Order ID: 324669444
Inserted row with Order ID: 246248090
Inserted row with Order ID: 485070693
Inserted row with Order ID: 573998582
Inserted row with Order ID: 116205585
Inserted row with Order ID: 135178029
Inserted row with Order ID: 824714744
Inserted row wit

data present in bronze level

In [13]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import SimpleStatement

# Connection setup
client_id = 'XTHZjgZRWbSZqXvWcYmtJAud'
client_secret = 'qh-ElzlZAG_bfeoDjZ9.ZCbht7jJ_WrQrh27Se6RsPNFxgz6wBnlY+B2KkyCXpwiMna.-F_bQy1I7Le5.CByML5,1MjuoWRNTjYYjl8NbrcwI1Yqrq2G2Q.u48XSY0Km'
secure_connect_bundle_path = '/content/Assignment3/secure-connect-sneha.zip'

# Set up authentication
auth_provider = PlainTextAuthProvider(client_id, client_secret)
cluster = Cluster(
    cloud={'secure_connect_bundle': secure_connect_bundle_path},
    auth_provider=auth_provider,
    protocol_version=4
)

try:
    session = cluster.connect()
    session.set_keyspace('sneha')

    # Query to select data from bronze_sales table
    query = SimpleStatement("SELECT * FROM bronze_sales LIMIT 10")
    rows = session.execute(query)

    # Print column names
    if rows:
        print("\nColumns:", rows.column_names)
        print("\nData:")
        for row in rows:
            print(row)

    # Get total count
    count_query = SimpleStatement("SELECT COUNT(*) FROM bronze_sales")
    count = session.execute(count_query).one()[0]
    print(f"\nTotal number of rows in bronze_sales: {count}")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    cluster.shutdown()
    print("\nConnection closed")


Columns: ['id', 'country', 'item_type', 'order_date', 'order_id', 'order_priority', 'region', 'sales_channel', 'ship_date', 'total_cost', 'total_profit', 'total_revenue', 'unit_cost', 'unit_price', 'units_sold']

Data:
Row(id=UUID('bbf70974-6e73-4439-a368-906c0f901633'), country='Antigua and Barbuda ', item_type='Office Supplies', order_date=Date(15249), order_id='286891067', order_priority='C', region='Central America and the Caribbean', sales_channel='Offline', ship_date=Date(15251), total_cost=Decimal('3305673.12'), total_profit=Decimal('794996.25'), total_revenue=Decimal('4100669.37'), unit_cost=Decimal('524.96'), unit_price=Decimal('651.21'), units_sold=6297)
Row(id=UUID('0f475459-74d4-40c1-aa80-8f1715f82d33'), country='Turkmenistan', item_type='Vegetables', order_date=Date(16538), order_id='116205585', order_priority='M', region='Asia', sales_channel='Online', ship_date=Date(16588), total_cost=Decimal('606503.1'), total_profit=Decimal('421077.1'), total_revenue=Decimal('1027580.

## creating silver level table

In [18]:
# Silver Table Creation Code
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from collections import defaultdict
from decimal import Decimal
import uuid

# Connection setup
client_id = 'XTHZjgZRWbSZqXvWcYmtJAud'
client_secret = 'qh-ElzlZAG_bfeoDjZ9.ZCbht7jJ_WrQrh27Se6RsPNFxgz6wBnlY+B2KkyCXpwiMna.-F_bQy1I7Le5.CByML5,1MjuoWRNTjYYjl8NbrcwI1Yqrq2G2Q.u48XSY0Km'
secure_connect_bundle_path = '/content/Assignment3/secure-connect-sneha.zip'

auth_provider = PlainTextAuthProvider(client_id, client_secret)
cluster = Cluster(
    cloud={'secure_connect_bundle': secure_connect_bundle_path},
    auth_provider=auth_provider,
    protocol_version=4
)

try:
    session = cluster.connect()
    session.set_keyspace('sneha')

    # Drop existing Silver table if it exists
    session.execute("DROP TABLE IF EXISTS silver_sales_summary")

    # Create Silver Table
    create_silver_table_query = """
    CREATE TABLE IF NOT EXISTS silver_sales_summary (
        id UUID PRIMARY KEY,
        region TEXT,
        item_type TEXT,
        total_units_sold INT,
        average_unit_price DECIMAL,
        average_unit_cost DECIMAL,
        total_revenue DECIMAL,
        total_profit DECIMAL
    )
    """
    session.execute(create_silver_table_query)
    print("Silver table created successfully")

    # Fetch data from Bronze table
    select_query = """
    SELECT region, item_type, units_sold, unit_price, unit_cost, total_revenue, total_profit
    FROM bronze_sales
    """
    rows = session.execute(select_query)

    # Aggregate data using Python
    aggregated_data = defaultdict(lambda: {
        'total_units_sold': 0,
        'total_revenue': Decimal(0),
        'total_profit': Decimal(0),
        'unit_price_sum': Decimal(0),
        'unit_cost_sum': Decimal(0),
        'count': 0
    })

    for row in rows:
        # Access data by index
        region = row[0]
        item_type = row[1]
        units_sold = row[2]
        unit_price = row[3]
        unit_cost = row[4]
        total_revenue = row[5]
        total_profit = row[6]

        key = (region, item_type)
        data = aggregated_data[key]
        data['total_units_sold'] += units_sold
        data['total_revenue'] += total_revenue
        data['total_profit'] += total_profit
        data['unit_price_sum'] += unit_price
        data['unit_cost_sum'] += unit_cost
        data['count'] += 1

    # Insert aggregated data into Silver table
    insert_query = """
    INSERT INTO silver_sales_summary (
        id, region, item_type, total_units_sold, average_unit_price,
        average_unit_cost, total_revenue, total_profit
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """

    for (region, item_type), data in aggregated_data.items():
        average_unit_price = data['unit_price_sum'] / Decimal(data['count'])
        average_unit_cost = data['unit_cost_sum'] / Decimal(data['count'])
        session.execute(insert_query, (
            uuid.uuid4(),
            region,
            item_type,
            data['total_units_sold'],
            average_unit_price,
            average_unit_cost,
            data['total_revenue'],
            data['total_profit']
        ))
        print(f"Inserted row for Region: {region}, Item_Type: {item_type}")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    cluster.shutdown()
    print("Connection closed")


Silver table created successfully
Inserted row for Region: Central America and the Caribbean, Item_Type: Office Supplies
Inserted row for Region: Asia, Item_Type: Vegetables
Inserted row for Region: Europe, Item_Type: Household
Inserted row for Region: Sub-Saharan Africa, Item_Type: Fruits
Inserted row for Region: Australia and Oceania, Item_Type: Baby Food
Inserted row for Region: Central America and the Caribbean, Item_Type: Household
Inserted row for Region: Sub-Saharan Africa, Item_Type: Clothes
Inserted row for Region: North America, Item_Type: Personal Care
Inserted row for Region: Sub-Saharan Africa, Item_Type: Meat
Inserted row for Region: Sub-Saharan Africa, Item_Type: Cosmetics
Inserted row for Region: Australia and Oceania, Item_Type: Beverages
Inserted row for Region: Sub-Saharan Africa, Item_Type: Personal Care
Inserted row for Region: Europe, Item_Type: Office Supplies
Inserted row for Region: Asia, Item_Type: Office Supplies
Inserted row for Region: Europe, Item_Type: Be

## Creating Gold level table
Gold sales by region

In [23]:
# Gold Table: gold_sales_by_region

from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from collections import defaultdict
from decimal import Decimal
import uuid

# Connection setup
client_id = 'XTHZjgZRWbSZqXvWcYmtJAud'
client_secret = 'qh-ElzlZAG_bfeoDjZ9.ZCbht7jJ_WrQrh27Se6RsPNFxgz6wBnlY+B2KkyCXpwiMna.-F_bQy1I7Le5.CByML5,1MjuoWRNTjYYjl8NbrcwI1Yqrq2G2Q.u48XSY0Km'
secure_connect_bundle_path = '/content/Assignment3/secure-connect-sneha.zip'

auth_provider = PlainTextAuthProvider(client_id, client_secret)
cluster = Cluster(
    cloud={'secure_connect_bundle': secure_connect_bundle_path},
    auth_provider=auth_provider,
    protocol_version=4
)
try:
    session = cluster.connect()
    session.set_keyspace('sneha')

    # Drop existing Gold table if it exists
    session.execute("DROP TABLE IF EXISTS gold_sales_by_region")

    # Create Gold Table
    create_gold_table_query = """
    CREATE TABLE IF NOT EXISTS gold_sales_by_region (
        region TEXT PRIMARY KEY,
        total_units_sold INT,
        total_revenue DECIMAL,
        total_profit DECIMAL
    )
    """
    session.execute(create_gold_table_query)
    print("Gold table (sales by region) created successfully")

    # Fetch data from Silver table
    select_query = """
    SELECT region, total_units_sold, total_revenue, total_profit
    FROM silver_sales_summary
    """
    rows = session.execute(select_query)

    # Aggregate data by region
    region_data = defaultdict(lambda: {'total_units_sold': 0, 'total_revenue': Decimal(0), 'total_profit': Decimal(0)})

    for row in rows:
        region_data[row.region]['total_units_sold'] += row.total_units_sold
        region_data[row.region]['total_revenue'] += row.total_revenue
        region_data[row.region]['total_profit'] += row.total_profit

    # Insert aggregated data into Gold table
    insert_query = """
    INSERT INTO gold_sales_by_region (
        region, total_units_sold, total_revenue, total_profit
    ) VALUES (%s, %s, %s, %s)
    """

    for region, data in region_data.items():
        session.execute(insert_query, (
            region,
            data['total_units_sold'],
            data['total_revenue'],
            data['total_profit']
        ))
        print(f"Inserted row for Region: {region}")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    cluster.shutdown()
    print("Connection closed")


Gold table (sales by region) created successfully
Inserted row for Region: Sub-Saharan Africa
Inserted row for Region: Australia and Oceania
Inserted row for Region: Middle East and North Africa
Inserted row for Region: Central America and the Caribbean
Inserted row for Region: Asia
Inserted row for Region: North America
Inserted row for Region: Europe
Connection closed


gold sales by items type

In [26]:
# Gold Table: gold_sales_by_item_type

from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from collections import defaultdict
from decimal import Decimal
import uuid

# Connection setup
client_id = 'XTHZjgZRWbSZqXvWcYmtJAud'
client_secret = 'qh-ElzlZAG_bfeoDjZ9.ZCbht7jJ_WrQrh27Se6RsPNFxgz6wBnlY+B2KkyCXpwiMna.-F_bQy1I7Le5.CByML5,1MjuoWRNTjYYjl8NbrcwI1Yqrq2G2Q.u48XSY0Km'
secure_connect_bundle_path = '/content/Assignment3/secure-connect-sneha.zip'

auth_provider = PlainTextAuthProvider(client_id, client_secret)
cluster = Cluster(
    cloud={'secure_connect_bundle': secure_connect_bundle_path},
    auth_provider=auth_provider,
    protocol_version=4
)
try:
    session = cluster.connect()
    session.set_keyspace('sneha')

    # Drop existing Gold table if it exists
    session.execute("DROP TABLE IF EXISTS gold_sales_by_item_type")

    # Create Gold Table
    create_gold_table_query = """
    CREATE TABLE IF NOT EXISTS gold_sales_by_item_type (
        item_type TEXT PRIMARY KEY,
        total_units_sold INT,
        total_revenue DECIMAL,
        total_profit DECIMAL,
        average_profit_margin DECIMAL
    )
    """
    session.execute(create_gold_table_query)
    print("Gold table (sales by item type) created successfully")

    # Fetch data from Silver table
    select_query = """
    SELECT item_type, total_units_sold, total_revenue, total_profit
    FROM silver_sales_summary
    """
    rows = session.execute(select_query)

    # Aggregate data by item_type
    item_type_data = defaultdict(lambda: {'total_units_sold': 0, 'total_revenue': Decimal(0), 'total_profit': Decimal(0)})

    for row in rows:
        item_type_data[row.item_type]['total_units_sold'] += row.total_units_sold
        item_type_data[row.item_type]['total_revenue'] += row.total_revenue
        item_type_data[row.item_type]['total_profit'] += row.total_profit

    # Insert aggregated data into Gold table
    insert_query = """
    INSERT INTO gold_sales_by_item_type (
        item_type, total_units_sold, total_revenue, total_profit, average_profit_margin
    ) VALUES (%s, %s, %s, %s, %s)
    """

    for item_type, data in item_type_data.items():
        # Calculate average profit margin
        average_profit_margin = (data['total_profit'] / data['total_revenue']) if data['total_revenue'] > 0 else Decimal(0)
        session.execute(insert_query, (
            item_type,
            data['total_units_sold'],
            data['total_revenue'],
            data['total_profit'],
            average_profit_margin
        ))
        print(f"Inserted row for Item_Type: {item_type}")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    cluster.shutdown()
    print("Connection closed")


Gold table (sales by item type) created successfully
Inserted row for Item_Type: Baby Food
Inserted row for Item_Type: Household
Inserted row for Item_Type: Beverages
Inserted row for Item_Type: Meat
Inserted row for Item_Type: Fruits
Inserted row for Item_Type: Cereal
Inserted row for Item_Type: Cosmetics
Inserted row for Item_Type: Personal Care
Inserted row for Item_Type: Office Supplies
Inserted row for Item_Type: Clothes
Inserted row for Item_Type: Vegetables
Inserted row for Item_Type: Snacks
Connection closed


Gold top performing items

In [29]:
# Gold Table: gold_top_performing_items

from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from collections import defaultdict
from decimal import Decimal
import uuid

# Connection setup
client_id = 'XTHZjgZRWbSZqXvWcYmtJAud'
client_secret = 'qh-ElzlZAG_bfeoDjZ9.ZCbht7jJ_WrQrh27Se6RsPNFxgz6wBnlY+B2KkyCXpwiMna.-F_bQy1I7Le5.CByML5,1MjuoWRNTjYYjl8NbrcwI1Yqrq2G2Q.u48XSY0Km'
secure_connect_bundle_path = '/content/Assignment3/secure-connect-sneha.zip'

auth_provider = PlainTextAuthProvider(client_id, client_secret)
cluster = Cluster(
    cloud={'secure_connect_bundle': secure_connect_bundle_path},
    auth_provider=auth_provider,
    protocol_version=4
)
try:
    session = cluster.connect()
    session.set_keyspace('sneha')

    # Drop existing Gold table if it exists
    session.execute("DROP TABLE IF EXISTS gold_top_performing_items")

    # Create Gold Table
    create_gold_table_query = """
    CREATE TABLE IF NOT EXISTS gold_top_performing_items (
        id UUID PRIMARY KEY,
        region TEXT,
        item_type TEXT,
        total_profit DECIMAL
    )
    """
    session.execute(create_gold_table_query)
    print("Gold table (top performing items) created successfully")

    # Fetch and rank data from Silver table
    select_query = """
    SELECT region, item_type, total_profit
    FROM silver_sales_summary
    """
    rows = session.execute(select_query)

    # Rank items by region and total profit
    ranked_data = defaultdict(list)
    for row in rows:
        ranked_data[row.region].append((row.item_type, row.total_profit))

    # Sort items within each region by profit (descending)
    for region, items in ranked_data.items():
        ranked_data[region] = sorted(items, key=lambda x: x[1], reverse=True)[:5]  # Top 5 items

    # Insert ranked data into Gold table
    insert_query = """
    INSERT INTO gold_top_performing_items (
        id, region, item_type, total_profit
    ) VALUES (%s, %s, %s, %s)
    """

    for region, items in ranked_data.items():
        for item_type, total_profit in items:
            session.execute(insert_query, (
                uuid.uuid4(),
                region,
                item_type,
                total_profit
            ))
            print(f"Inserted Top Item for Region: {region}, Item_Type: {item_type}")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    cluster.shutdown()
    print("Connection closed")


Gold table (top performing items) created successfully
Inserted Top Item for Region: Sub-Saharan Africa, Item_Type: Cosmetics
Inserted Top Item for Region: Sub-Saharan Africa, Item_Type: Office Supplies
Inserted Top Item for Region: Sub-Saharan Africa, Item_Type: Clothes
Inserted Top Item for Region: Sub-Saharan Africa, Item_Type: Household
Inserted Top Item for Region: Sub-Saharan Africa, Item_Type: Meat
Inserted Top Item for Region: Australia and Oceania, Item_Type: Household
Inserted Top Item for Region: Australia and Oceania, Item_Type: Clothes
Inserted Top Item for Region: Australia and Oceania, Item_Type: Baby Food
Inserted Top Item for Region: Australia and Oceania, Item_Type: Cereal
Inserted Top Item for Region: Australia and Oceania, Item_Type: Beverages
Inserted Top Item for Region: Middle East and North Africa, Item_Type: Cosmetics
Inserted Top Item for Region: Middle East and North Africa, Item_Type: Office Supplies
Inserted Top Item for Region: Middle East and North Africa